In [3]:
#!pip install tensorflow

In [1]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.utils import to_categorical as y_cat
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from tqdm import tqdm
import random
import pickle

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.24.3
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [9]:
##  Load the datasets (CIFAR10):

cifar = tf.keras.datasets.cifar10 

(x_tr, y_tr_cat) , (x_te,y_te_cat) = cifar.load_data() 

x_tr = x_tr / 255.0
x_te = x_te / 255.0

y_tr = y_cat(y_tr_cat)
y_te = y_cat(y_te_cat)

170498071/170498071 [==============================] - 3s 0us/step


In [2]:
inp_shape = (32,32,3)
initializer = tf.keras.initializers.HeNormal(seed = 10)

def model_original(inp_shape):
    model = tf.keras.models.Sequential()
    model.add(tf.keras.layers.Conv2D(64, (3,3), activation = 'relu', kernel_initializer= initializer, padding = 'same', input_shape = inp_shape))
    model.add(tf.keras.layers.Conv2D(64, (3,3), activation = 'relu', kernel_initializer= initializer, padding = 'same'))
    model.add(tf.keras.layers.MaxPooling2D((2,2)))
    model.add(tf.keras.layers.Dropout(0.4))
    model.add(tf.keras.layers.Conv2D(128, (3,3), activation = 'relu', kernel_initializer= initializer, padding = 'same'))
    model.add(tf.keras.layers.Conv2D(128, (3,3), activation = 'relu', kernel_initializer= initializer, padding = 'same'))
    model.add(tf.keras.layers.MaxPooling2D((2,2)))
    model.add(tf.keras.layers.Dropout(0.4))
    model.add(tf.keras.layers.Flatten())
    model.add(tf.keras.layers.Dense(1024, kernel_initializer = initializer, activation = 'relu'))
    model.add(tf.keras.layers.Dense(1024, kernel_initializer = initializer, activation = 'relu'))
    model.add(tf.keras.layers.Dense(10, kernel_initializer = initializer, activation = 'softmax'))
    return model

In [16]:
## Forget Phase:

orig_model = tf.keras.models.load_model('/kaggle/input/cifar10/Original_CIFAR10.h5')

cifar_10_orig = orig_model

In [10]:
cifar_10_orig.evaluate(x_te, y_te)

313/313 [==============================] - 11s 4ms/step - loss: 1.3573 - accuracy: 0.7723


[1.357301115989685, 0.7723000049591064]

In [17]:
def model_st(inp_shape):
    model = tf.keras.models.Sequential()
    model.add(tf.keras.layers.Conv2D(64, (3,3), activation = 'relu', kernel_initializer= initializer, padding = 'same', input_shape = inp_shape))
    model.add(tf.keras.layers.Conv2D(64, (3,3), activation = 'relu', kernel_initializer= initializer, padding = 'same'))
    model.add(tf.keras.layers.MaxPooling2D((2,2)))
    model.add(tf.keras.layers.Dropout(0.4))
    model.add(tf.keras.layers.Flatten())
    model.add(tf.keras.layers.Dense(1024, kernel_initializer = initializer, activation = 'relu'))
    model.add(tf.keras.layers.Dense(1024, kernel_initializer = initializer, activation = 'relu'))
    model.add(tf.keras.layers.Dense(10, kernel_initializer = initializer, activation = 'softmax'))
    return model

stochastic_teacher_model = model_st((32,32,3))
stochastic_teacher_model.compile(optimizer = 'adam',loss = 'categorical_crossentropy', metrics = ['accuracy'])

In [18]:
def unlearn_loss(y_act, y_pred):
  return tf.keras.losses.kullback_leibler_divergence(y_act, y_pred)
eval_orig = []
eval_unlearn = []
eval_sto = []
eval_ftn = []
b_size = 64

In [19]:
for i in range(10):
    forget_cls = i

    ## Define the unlearning model:
    unlearned_model = tf.keras.models.clone_model(cifar_10_orig)  
    unlearned_model.set_weights(cifar_10_orig.get_weights())
    unlearned_model.compile(optimizer = 'adam',loss = unlearn_loss, metrics = ['accuracy'])


    df_ind_te = [i for i in range(y_te.shape[0]) if y_te_cat[i] == forget_cls]
    dr_ind_te = [i for i in range(y_te.shape[0]) if y_te_cat[i] != forget_cls]

    df_ind_tr = [i for i in range(y_tr.shape[0]) if y_tr_cat[i] == forget_cls]
    dr_ind_tr = [i for i in range(y_tr.shape[0]) if y_tr_cat[i] != forget_cls]

    dr_x_te, dr_y_te = x_te[dr_ind_te], y_te[dr_ind_te]
    df_x_te, df_y_te = x_te[df_ind_te], y_te[df_ind_te]

    dr_x_tr, dr_y_tr = x_tr[dr_ind_tr], y_tr[dr_ind_tr]
    df_x_tr, df_y_tr = x_tr[df_ind_tr], y_tr[df_ind_tr]

    evaluations_o = unlearned_model.evaluate(x_te, y_te, verbose = 0), unlearned_model.evaluate(dr_x_te, dr_y_te, verbose = 0), unlearned_model.evaluate(df_x_te, df_y_te, verbose = 0)
    eval_orig.append(evaluations_o)

    evaluations_st = stochastic_teacher_model.evaluate(x_te, y_te, verbose = 0), stochastic_teacher_model.evaluate(dr_x_te, dr_y_te, verbose = 0), stochastic_teacher_model.evaluate(df_x_te, df_y_te, verbose = 0)
    eval_sto.append(evaluations_st)

    ## Forget Phase:

    STM_pred = stochastic_teacher_model.predict(df_x_tr, verbose = 0)
    unlearned_model.fit(df_x_tr, STM_pred, epochs = 100, batch_size = b_size, verbose = 0)

    ## Fine Tune Phase:
    orig_pred = cifar_10_orig.predict(dr_x_tr)
    unlearned_model.fit(dr_x_tr, orig_pred, epochs = 1, batch_size = b_size, verbose = 0)

    evaluations_un = unlearned_model.evaluate(x_te, y_te, verbose = 0), unlearned_model.evaluate(dr_x_te, dr_y_te, verbose = 0), unlearned_model.evaluate(df_x_te, df_y_te, verbose = 0)
    eval_unlearn.append(evaluations_un)

    ## Simple Unlearning by Fine Tuning:

    fn_model = tf.keras.models.clone_model(cifar_10_orig)
    fn_model.set_weights(cifar_10_orig.get_weights())

    fn_model.compile(optimizer = 'adam',loss = 'categorical_crossentropy', metrics = ['accuracy'])
    fn_model.fit(dr_x_tr, dr_y_tr, epochs = 1, batch_size = b_size, verbose = 0)

    evaluations_rt = fn_model.evaluate(x_te, y_te, verbose = 0), fn_model.evaluate(dr_x_te, dr_y_te, verbose = 0), fn_model.evaluate(df_x_te, df_y_te, verbose = 0)
    eval_ftn.append(evaluations_rt)
    print("Forget Class: ", forget_cls, "Orig_eval: ", evaluations_o, "Unlearn Eval: ", eval_unlearn, "FineTune_eval:", evaluations_rt)

2023-12-10 07:12:48.866267: E tensorflow/core/grappler/optimizers/meta_optimizer.cc:954] layout failed: INVALID_ARGUMENT: Size of values 0 does not match size of permutation 4 @ fanin shape insequential/dropout/dropout/SelectV2-2-TransposeNHWCToNCHW-LayoutOptimizer


1407/1407 [==============================] - 3s 2ms/step


2023-12-10 07:15:02.392890: E tensorflow/core/grappler/optimizers/meta_optimizer.cc:954] layout failed: INVALID_ARGUMENT: Size of values 0 does not match size of permutation 4 @ fanin shape insequential/dropout/dropout/SelectV2-2-TransposeNHWCToNCHW-LayoutOptimizer


Forget Class:  0 Orig_eval:  ([1.3008519411087036, 0.7723000049591064], [1.2714710235595703, 0.776888906955719], [1.5652744770050049, 0.7310000061988831]) Unlearn Eval:  [([1.4894161224365234, 0.7023000121116638], [0.9684965014457703, 0.7721111178398132], [6.177692413330078, 0.07400000095367432])] FineTune_eval: ([1.806422472000122, 0.7458000183105469], [1.4598275423049927, 0.7825555801391602], [4.925783634185791, 0.41499999165534973])


2023-12-10 07:15:25.622430: E tensorflow/core/grappler/optimizers/meta_optimizer.cc:954] layout failed: INVALID_ARGUMENT: Size of values 0 does not match size of permutation 4 @ fanin shape insequential/dropout/dropout/SelectV2-2-TransposeNHWCToNCHW-LayoutOptimizer


...

Forget Class:  8 Orig_eval:  ([1.3008519411087036, 0.7723000049591064], [1.3547792434692383, 0.7598888874053955], [0.8155019879341125, 0.8840000033378601]) Unlearn Eval:  [([1.4894161224365234, 0.7023000121116638], [0.9684965014457703, 0.7721111178398132], [6.177692413330078, 0.07400000095367432]), ([1.4194254875183105, 0.7056000232696533], [1.0324265956878662, 0.7557777762413025], [4.902415752410889, 0.2540000081062317]), ([1.3548154830932617, 0.7196000218391418], [0.8773205876350403, 0.788777768611908], [5.6522674560546875, 0.09700000286102295]), ([1.271409511566162, 0.7310000061988831], [0.8035393357276917, 0.8072222471237183], [5.482238292694092, 0.04500000178813934]), ([1.3975926637649536, 0.7071999907493591], [0.8980613350868225, 0.7814444303512573], [5.893380641937256, 0.039000000804662704]), ([1.3896527290344238, 0.7189000248908997], [0.8485968708992004, 0.7941111326217651], [6.259160041809082, 0.041999999433755875]), ([1.2630573511123657, 0.7117000222206116], [0.98292750120162

2023-12-10 07:36:24.485626: E tensorflow/core/grappler/optimizers/meta_optimizer.cc:954] layout failed: INVALID_ARGUMENT: Size of values 0 does not match size of permutation 4 @ fanin shape insequential/dropout/dropout/SelectV2-2-TransposeNHWCToNCHW-LayoutOptimizer


1407/1407 [==============================] - 3s 2ms/step


2023-12-10 07:38:35.507045: E tensorflow/core/grappler/optimizers/meta_optimizer.cc:954] layout failed: INVALID_ARGUMENT: Size of values 0 does not match size of permutation 4 @ fanin shape insequential/dropout/dropout/SelectV2-2-TransposeNHWCToNCHW-LayoutOptimizer


Forget Class:  9 Orig_eval:  ([1.3008519411087036, 0.7723000049591064], [1.3588531017303467, 0.7608888745307922], [0.7788363695144653, 0.875]) Unlearn Eval:  [([1.4894161224365234, 0.7023000121116638], [0.9684965014457703, 0.7721111178398132], [6.177692413330078, 0.07400000095367432]), ([1.4194254875183105, 0.7056000232696533], [1.0324265956878662, 0.7557777762413025], [4.902415752410889, 0.2540000081062317]), ([1.3548154830932617, 0.7196000218391418], [0.8773205876350403, 0.788777768611908], [5.6522674560546875, 0.09700000286102295]), ([1.271409511566162, 0.7310000061988831], [0.8035393357276917, 0.8072222471237183], [5.482238292694092, 0.04500000178813934]), ([1.3975926637649536, 0.7071999907493591], [0.8980613350868225, 0.7814444303512573], [5.893380641937256, 0.039000000804662704]), ([1.3896527290344238, 0.7189000248908997], [0.8485968708992004, 0.7941111326217651], [6.259160041809082, 0.041999999433755875]), ([1.2630573511123657, 0.7117000222206116], [0.9829275012016296, 0.7641111

In [25]:
for i in range(len(eval_orig)):
    te, rt, f = eval_orig[i]
    print("Original Model")
    print(i, np.round(te[1], 2), np.round(rt[1], 2), np.round(f[1], 2))
    te, rt, f = eval_unlearn[i]
    print("Unlearn Model")
    print(i, np.round(te[1], 2), np.round(rt[1], 2), np.round(f[1],2))
    te, rt, f = eval_ftn[i]
    print("Fine Tuned Model")
    print(i, np.round(te[1], 2), np.round(rt[1], 2), np.round(f[1], 2))

Original Model
0 0.77 0.78 0.73
Unlearn Model
0 0.7 0.77 0.07
Fine Tuned Model
0 0.75 0.78 0.41
Original Model
1 0.77 0.76 0.88
Unlearn Model
1 0.71 0.76 0.25
Fine Tuned Model
1 0.74 0.77 0.49
Original Model
2 0.77 0.78 0.69
Unlearn Model
2 0.72 0.79 0.1
Fine Tuned Model
2 0.74 0.8 0.23
Original Model
3 0.77 0.79 0.63
Unlearn Model
3 0.73 0.81 0.05
Fine Tuned Model
3 0.74 0.81 0.15
Original Model
4 0.77 0.78 0.74
Unlearn Model
4 0.71 0.78 0.04
Fine Tuned Model
4 0.73 0.8 0.18
Original Model
5 0.77 0.78 0.67
Unlearn Model
5 0.72 0.79 0.04
Fine Tuned Model
5 0.74 0.8 0.16
Original Model
6 0.77 0.76 0.85
Unlearn Model
6 0.71 0.76 0.24
Fine Tuned Model
6 0.76 0.78 0.52
Original Model
7 0.77 0.77 0.78
Unlearn Model
7 0.7 0.76 0.11
Fine Tuned Model
7 0.75 0.77 0.5
Original Model
8 0.77 0.76 0.88
Unlearn Model
8 0.7 0.76 0.19
Fine Tuned Model
8 0.75 0.77 0.54
Original Model
9 0.77 0.76 0.88
Unlearn Model
9 0.72 0.76 0.36
Fine Tuned Model
9 0.75 0.77 0.54


In [36]:
orig_model_te = []
orig_model_f = []
orig_model_re = []

un_model_te = []
un_model_f = []
un_model_re = []

ft_model_te = []
ft_model_f = []
ft_model_re = []

diff_un_orig_te = []
diff_un_orig_re = []
diff_un_orig_f = []

diff_un_ft_te = []
diff_un_ft_re = []
diff_un_ft_f = []


for i in range(len(eval_orig)):
    te, rt, f = eval_orig[i]
    orig_model_te.append(te[1])
    orig_model_re.append(rt[1])
    orig_model_f.append(f[1])
    
    print(orig_model_f)

for i in range(len(eval_unlearn)):
    te, rt, f = eval_unlearn[i]
    un_model_te.append(te[1])
    un_model_re.append(rt[1])
    un_model_f.append(f[1])
    
    te1, rt1, f1 = eval_orig[i]
    
    diff_un_orig_te.append(abs(te1[1]-te[1]))
    diff_un_orig_re.append(abs(rt1[1]-rt[1]))
    diff_un_orig_f.append(abs(f1[1]-f[1]))

for i in range(len(eval_unlearn)):
    te, rt, f = eval_unlearn[i]
    
    te1, rt1, f1 = eval_ftn[i]
    
    ft_model_te.append(te1[1])
    ft_model_re.append(rt1[1])
    ft_model_f.append(f1[1])
    
    diff_un_ft_te.append(abs(te1[1]-te[1]))
    diff_un_ft_re.append(abs(rt1[1]-rt[1]))
    diff_un_ft_f.append(abs(f1[1]-f[1]))

[0.7310000061988831]
[0.7310000061988831, 0.8790000081062317]
[0.7310000061988831, 0.8790000081062317, 0.6850000023841858]
[0.7310000061988831, 0.8790000081062317, 0.6850000023841858, 0.628000020980835]
[0.7310000061988831, 0.8790000081062317, 0.6850000023841858, 0.628000020980835, 0.7419999837875366]
[0.7310000061988831, 0.8790000081062317, 0.6850000023841858, 0.628000020980835, 0.7419999837875366, 0.6729999780654907]
[0.7310000061988831, 0.8790000081062317, 0.6850000023841858, 0.628000020980835, 0.7419999837875366, 0.6729999780654907, 0.8450000286102295]
[0.7310000061988831, 0.8790000081062317, 0.6850000023841858, 0.628000020980835, 0.7419999837875366, 0.6729999780654907, 0.8450000286102295, 0.781000018119812]
[0.7310000061988831, 0.8790000081062317, 0.6850000023841858, 0.628000020980835, 0.7419999837875366, 0.6729999780654907, 0.8450000286102295, 0.781000018119812, 0.8840000033378601]
[0.7310000061988831, 0.8790000081062317, 0.6850000023841858, 0.628000020980835, 0.7419999837875366,

In [39]:
acc_print(orig_model_f)

0.7723000049591064

In [40]:
orig_model_f

[0.7310000061988831,
 0.8790000081062317,
 0.6850000023841858,
 0.628000020980835,
 0.7419999837875366,
 0.6729999780654907,
 0.8450000286102295,
 0.781000018119812,
 0.8840000033378601,
 0.875]

In [33]:
def acc_print(x):
    return np.mean(np.array(x))

In [35]:
print("Original Model mean accuracy On test:", acc_print(orig_model_te))
print("Original Model mean accuracy On retain:", acc_print(orig_model_re))
print("Original Model mean accuracy On forget:", acc_print(orig_model_f))

print()

print("Unlearned Model mean accuracy On test:", acc_print(un_model_te))
print("Unlearned Model mean accuracy On retain:", acc_print(un_model_re))
print("Unlearned Model mean accuracy On forget:", acc_print(un_model_f))

print()

print("Fine Tuned Model mean accuracy On test:", acc_print(ft_model_te))
print("Fine Tuned Model mean accuracy On retain:", acc_print(ft_model_re))
print("Fine Tuned Model mean accuracy On forget:", acc_print(ft_model_f))

print()

print("Mean Difference Orig and Unlearn On test:", acc_print(diff_un_orig_te))
print("Mean Difference Orig and Unlearn On retain:", acc_print(diff_un_orig_re))
print("Mean Difference Orig and Unlearn On forget:", acc_print(diff_un_orig_f))

print()

print("Mean Difference Orig and FTN On test:", acc_print(diff_un_ft_te))
print("Mean Difference Orig and FTN On retain:", acc_print(diff_un_ft_re))
print("Mean Difference Orig and FTN On forget:", acc_print(diff_un_ft_f))

Original Model mean accuracy On test: 0.7723000049591064
Original Model mean accuracy On retain: 0.7723000049591064
Original Model mean accuracy On forget: 0.7723000049591064

Unlearned Model mean accuracy On test: 0.711680006980896
Unlearned Model mean accuracy On retain: 0.7746333360671998
Unlearned Model mean accuracy On forget: 0.145100000500679

Fine Tuned Model mean accuracy On test: 0.7441799998283386
Fine Tuned Model mean accuracy On retain: 0.7854555606842041
Fine Tuned Model mean accuracy On forget: 0.3727000042796135

Mean Difference Orig and Unlearn On test: 0.06061999797821045
Mean Difference Orig and Unlearn On retain: 0.006355559825897217
Mean Difference Orig and Unlearn On forget: 0.6272000044584274

Mean Difference Orig and FTN On test: 0.03249999284744263
Mean Difference Orig and FTN On retain: 0.011066675186157227
Mean Difference Orig and FTN On forget: 0.22760000377893447
